# Homework 4: Practical analysis with BioPython

For the homework, you are going to extend the code from the analysis of our FASTQ file in lectures 8 and 9.
Recall that the FASTQ file contains reads from a real sequencing run of influenza virus HA and NA genes.

The _actual sequences_ are as follows:

    5'-[end of HA]-AGGCGGCCGC-[16 X N barcode]-3'
    
or 

    5'-[end of NA]-AGGCGGCCGC-[16 X N barcode]-3'
    
The end of NA is:

    ...CACGATAGATAAATAATAGTGCACCAT
    
The end of HA is:

    ...CCGGATTTGCATATAATGATGCACCAT
    
The _sequencing reads_ (located in the FASTQ file) are from the reverse end of these actual sequences, so the first thing in the sequencing reads is the reverse complement of the barcode followed by the reverse complement of the constant sequence, etc.
The reads can originate from **either** HA or NA, and that will be distinguished by the most 3' end of the read.
But in our example exercise in class, we did not distinguish among reads matching to HA and NA, as we didn't even look far enough into the read to tell the identity.

For the homework, your goal is to write code that extends the material from lectures 8 and 9 to also distinguish between HA and NA.
This homework can be completed almost entirely by re-using code from lecture 9. You will need to set up your analysis to do the following:
 1. Get the reverse complement of each read.
 2. Determine if it matches the expected pattern for HA and NA, and if so which one.
 3. If it matches, extract the barcode and add it to a dictionary to keep track of counts.
 4. Determine the number and distribution of barcodes for HA and NA separately.

Please include code to address each of the following questions. Please include code comments to explain what your code is attempting to accomplish. Don't forget to include references to the sources you used to obtain your answer, including your classmates (if you are working in groups).  

1. How many reads map to HA, and how many reads map to NA?

In [55]:
### Import packages
import os
import pandas as pd
import numpy as np
import re

### Define Parameters
NA_end = 'CACGATAGATAAATAATAGTGCACCAT'
HA_end = 'CCGGATTTGCATATAATGATGCACCAT'
spacer = 'AGGCGGCCGC'
barcode_length = 16

### Define functions
# Iterate over all key-value pairs in dictionary 
# Adapted from https://www.studytonight.com/ 2023-10-19
def reverseComplement(str):    
    char_to_replace = {'A': 't', 'C': 'g', 'G': 'c', 'T':'a'}
    for key, value in char_to_replace.items():
        # Replace key character with value character in string
        str = str.replace(key, value)
    return(str[::-1].upper())

def read_fastq(path):
    ### Read in .fastq
    fastq_tbl = pd.read_table(path, header=None)
    
    ### Used chat GPT To troubleshoot this part using the prompt:
    # "How do I use pd.assign() to add a new column to a dataframe that contains the vector c("line", "new", "here", "to") repeating over the length of the dataframe"
    # https://pandas.pydata.org/Pandas_Cheat_Sheet.pdf
    fastq_line_headers = np.array(['name', 'seq', 'empty', 'q'])

    # Create index for each read
    n_reads = len(fastq_tbl)
    reads = np.array(range(0,int(n_reads/4), 1))
    reads.repeat(4)

    # Reformat .fastq as .table
    fastq_tbl['key'] = np.tile(fastq_line_headers, (len(fastq_tbl) // len(fastq_line_headers)) + 1)[:len(fastq_tbl)]
    fastq_tbl['read_index'] = reads.repeat(4)
    fastq_tbl = fastq_tbl.rename(columns={0:"value", 1:"key"})
    fastq_tbl = fastq_tbl.pivot(index='read_index', columns='key', values='value')

    # return fastq_table
    return(fastq_tbl)


def contains_seq(subject, query):
    for i in range(0,int(len(query)), 1):
        return(subject in query)


# Convert ends to reverse complement
NA_end_revcom = reverseComplement(NA_end)
HA_end_revcom = reverseComplement(HA_end)

# Read fastq file
reads_tbl = read_fastq(path = "../tfcb-homework04/barcodes_R1.fastq")

# Search for NA and HA sequences in fastq lines
reads_tbl['contains_NA'] = [NA_end_revcom in i for i in reads_tbl['seq']]
reads_tbl['contains_HA'] = [HA_end_revcom in i for i in reads_tbl['seq']]

# Help from
# https://pandas.pydata.org/docs/reference/api/pandas.Series.to_dict.html
NA_dict = pd.Series.to_dict(reads_tbl.loc[reads_tbl.contains_NA].seq)
HA_dict = pd.Series.to_dict(reads_tbl.loc[reads_tbl.contains_HA].seq)

n_NA = len(NA_dict)
n_HA = len(HA_dict)

print(str(n_NA) + ' reads map to NA.')
print(str(n_HA) + ' reads map to HA.')

4122 reads map to NA.
5409 reads map to HA.
ATGGTGCATCATTATATGCAAATCCGG


2. What is the HA barcode with the most counts (and how many counts)? Also answer the same question for NA.

    _Hint: you will need to find the key associated with the maximum value in your dictionary. There are many ways to do this._

In [108]:
def extract_barcode(df, col, new_col_name, pattern_spacer, barcode_length):
    pattern = re.compile('(?P<name>.'+ pattern_spacer + '[ACGT]{' + str(insert) + '}'+ "" +')')
    colList = df[col].tolist()
    # Help with this line was from ChatGPT with the erroring code below and the error.
    # ChatGPT identified that I needed to initialize the list as a blank list
    new_col = [None] * len(colList)

    for i in range(len(colList)):
        match = re.search(pattern, colList[i])
        if match == None:
            new_col[i] = None
        else:
            new_col[i] = match.group('name').replace(pattern_spacer, "")
    
    df[new_col_name] = new_col

    return(df)

# Run for NA spacer
extract_barcode(df = reads_tbl,
                col = "seq",
                new_col_name = "NA_barcodes",
                pattern_spacer = reverseComplement(spacer) + reverseComplement(NA_end),
                barcode_length = 16
                )

# Run for HA spacer
extract_barcode(df = reads_tbl,
                col = "seq",
                new_col_name = "HA_barcodes",
                pattern_spacer = reverseComplement(spacer) + reverseComplement(HA_end),
                barcode_length = 16
                )

# Assign barcodes
max_NA_barcode = reads_tbl.groupby('NA_barcodes').size().sort_values(ascending=False).head(1)
max_HA_barcode = reads_tbl.groupby('HA_barcodes').size().sort_values(ascending=False).head(1)

# Print barcodes
print(max_NA_barcode)
print("")
print(max_HA_barcode)


NA_barcodes
AAAAGGGAGTTCTGCTC    1271
dtype: int64

HA_barcodes
TCATTGCAAGGAGCCGT    1464
dtype: int64
